## Step 2: (Fake) Data! 
We're going to use the code below to pre-populate our kafka "messages" topic with fake customer requests. The code below **randomizes** customer IDs so your output will **differ slightly** from mine.

## Environment Setup
You know what to do....

In [ ]:
!pip install -U -r requirements.txt

In [ ]:
import json
import os
import random
import time
from kafka import KafkaProducer

In [ ]:
KAFKA_BOOTSTRAP_SERVER = os.environ.get('KAFKA_BOOTSTRAP_SERVER')
KAFKA_SECURITY_PROTOCOL = os.environ.get('KAFKA_SECURITY_PROTOCOL')
KAFKA_SASL_MECHANISM = os.environ.get('KAFKA_SASL_MECHANISM')
KAFKA_USERNAME = os.environ.get('KAFKA_USERNAME')
KAFKA_PASSWORD = os.environ.get('KAFKA_PASSWORD')

KAFKA_TOPIC_NOTEBOOKS = 'messages'

## Pre-populate messages
Similarly to the last notebook, we're using the `KafkaProducer` function to connect to Kafka. Here we simply extract some customer statements from an included file, assign an identifier, and drop the pair in the queue.

In [ ]:
producer = KafkaProducer(bootstrap_servers=[KAFKA_BOOTSTRAP_SERVER],
                            security_protocol=KAFKA_SECURITY_PROTOCOL,
                            sasl_mechanism=KAFKA_SASL_MECHANISM,
                            sasl_plain_username=KAFKA_USERNAME,
                            sasl_plain_password=KAFKA_PASSWORD,
                            api_version_auto_timeout_ms=30000,
                            max_block_ms=900000,
                            request_timeout_ms=450000,
                            acks='all')

with open('support/testdata1.csv') as csv:
    for message in csv:
        jsonpayload = json.dumps(
            {'customer_number': random.randint(1, 1000), 
             'txt': message.rsplit(',', 1)[0],
            }
        )
        producer.send(KAFKA_TOPIC_NOTEBOOKS, jsonpayload.encode('utf-8'))

producer.flush() # Important, especially if message size is small

## Next
You should see a ton of of messages in your [consumer notebook](0_kafka_consumer.ipynb). 

Next, let's use [3_trino_explore.ipynb](3_trino_explore.ipynb) to explore our data! 